In [1]:
import tensorflow as tf
tf.__version__

'2.18.0'

#  케라스로 RNN 구현하기

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

In [3]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(2, 10))) # features=2d, batch_size=None, output_dim=3(hidden_unit)
# model.add(SimpleRNN(3, input_length=2, input_dim=10)) 과 동일; input_length == timesteps
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 3)                   │              42 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 42 (168.00 B)

 Trainable params: 42 (168.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model = Sequential()
#model.add(SimpleRNN(3, batch_input_shape=(8,2,10))) # features=2d, batch_size=8, output_dim=3(hidden_unit)
#model.add(SimpleRNN(3, input_shape=(2, 10), batch_size=8))
# If you need to specify the batch size, do it as a separate argument to the add method.
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


ValueError: Unrecognized keyword arguments passed to SimpleRNN: {'batch_size': 8}

# 파이썬으로 RNN 구현하기

In [5]:
import numpy as np

timesteps = 10 # 시점의 수. NLP에서는 보통 문장의 길이가 된다.
input_dim = 4 # 입력의 차원. NLP에서는 보통 단어 벡터의 차원이 된다
hidden_size = 8 # 은닉 상태의 크기. 메모리 셀의 용량이다.

inputs = np.random.random((timesteps, input_dim)) # 입력에 해당되는 2D 텐서

# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듦.
hidden_state_t = np.zeros((hidden_size,)) # 초기 은닉 상태는 0(벡터)로 초기화

In [6]:
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듦. #8의 크기를 가지는 은닉 상태.
# 현재는 초기 은닉 상태로, 모든 차원이 0의 값을 가짐
print(hidden_state_t)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [8]:
Wx = np.random.random((hidden_size, input_dim)) # (8, 4) 크기의 2D 텐서 생성. 입력에 대한 가중치
Wh = np.random.random((hidden_size, hidden_size)) # (8, 8) 크기의 2D 텐서 생성. 은닉 상태에 대한 가중치
b = np.random.random((hidden_size)) # (8,) 크기의 1D 텐서 생성. 이 값은 편향(bias)

print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


In [14]:
print(Wx)
print(inputs[0])
print(np.dot(Wx, inputs[0]))

[[0.34802054 0.01532041 0.66773545 0.54060164]
 [0.07372371 0.70292518 0.37277498 0.48523213]
 [0.56079789 0.87295102 0.89638851 0.36149273]
 [0.19761632 0.57958787 0.95163049 0.82886251]
 [0.55336884 0.69708923 0.05929917 0.650993  ]
 [0.3918214  0.70794461 0.87495962 0.29215291]
 [0.75800724 0.61049087 0.58362295 0.52498714]
 [0.56626823 0.06088754 0.60123657 0.51181072]]
[0.76355158 0.50949225 0.02392841 0.94500128]
[0.80038436 0.88189169 1.23602015 1.25223293 1.39429538 0.95688942
 1.39989669 0.96144516]


In [25]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값이 입력됨
    output_t = np.tanh(np.dot(Wx, input_t) + np.dot(Wh, hidden_state_t) + b) # (8x4)(4x1) + (8x8)(8x1) + (8,) = (8,)
    total_hidden_states.append(list(output_t)) # 각 시점의 은닉 상태의 값을 계속해서 축적; (10, 8)
    print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 출력의 크기는 (timestep, output_dim)
    hidden_state_t = output_t

print(total_hidden_states)
total_hidden_states = np.stack(total_hidden_states, axis=0) # 출력 시간을 stack으로 정리; ?이게 꼭 필요할까?
print(total_hidden_states)

(1, 8)
(2, 8)
(3, 8)
(4, 8)
(5, 8)
(6, 8)
(7, 8)
(8, 8)
(9, 8)
(10, 8)
[[0.9999742506821069, 0.9999693064592718, 0.9999933044700724, 0.999789037916385, 0.999963847322731, 0.9996830157727042, 0.9999974798848305, 0.9999629258511291], [0.9999692310081306, 0.9999587134602981, 0.9999847283954107, 0.9997517222141917, 0.9999017504328276, 0.9994578037811374, 0.9999928923548491, 0.9999386819865861], [0.9999896950354488, 0.9999786937620898, 0.9999977117096123, 0.9999393013345915, 0.9999528113799785, 0.9998968451733956, 0.9999985646273841, 0.9999825575484205], [0.9999777846916758, 0.9999532479487159, 0.9999916980122341, 0.9997357871934044, 0.9999446338075022, 0.9996328861170368, 0.9999971175323589, 0.9999685039453511], [0.9999701514488081, 0.9999746815464005, 0.9999980684414025, 0.9998075588442348, 0.9999544671277664, 0.9998976817975128, 0.9999984543316492, 0.9999613115543821], [0.9999572517617243, 0.9999250145444009, 0.9999867931634112, 0.9994108590573492, 0.9998475248380441, 0.9995029984299337,

# 깊은 순환 신경망 (Deep Recurrent Neural Network)

In [19]:
model = Sequential()
#model.add(SimpleRNN(hidden_size, input_length=10, input_dim=5, return_sequences=True))
model.add(SimpleRNN(hidden_size, input_shape=(timesteps, input_dim), return_sequences=True)) # input_shape=(timesteps, input_dim)
model.add(SimpleRNN(hidden_size, return_sequences=True))
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn_3 (SimpleRNN)             │ (None, 10, 8)               │             112 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_4 (SimpleRNN)             │ (None, 10, 8)               │             136 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 248 (992.00 B)

 Trainable params: 248 (992.00 B)

 Non-trainable params: 0 (0.00 B)

# 양방향 순환 신경망 (Bidirectional Recurrent Neural Network)

In [20]:
from tensorflow.keras.layers import Bidirectional

In [22]:
timesteps = 10
input_dim = 5

model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences=True), input_shape=(timesteps, input_dim)))
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ (None, 10, 16)              │             224 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 224 (896.00 B)

 Trainable params: 224 (896.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences=True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences=True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequences=True)))
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_1 (Bidirectional)      │ (None, 10, 16)              │             224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 10, 16)              │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 10, 16)              │             400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_4 (Bidirectional)      │ (None, 10, 16)              │             400 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,424 (5.56 KB)

 Trainable params: 1,424 (5.56 KB)

 Non-trainable params: 0 (0.00 B)